In [1]:
!pip install keras-tuner --upgrade
%matplotlib inline
# Import our dependencies
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_blobs
import pandas as pd
from sklearn import __SKLEARN_SETUP__
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.utils import np_utils
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
from google.colab import drive
drive.mount('/content/drive')
csv_path = '/content/drive/MyDrive/UTSA_Homework/Resources/charity_data.csv'

#  Import and read the charity_data.csv.
application_df = pd.read_csv(csv_path)
application_df.head()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.3/167.3 KB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 16.6 MB/s eta 0:00:00
Mounted at /content/drive


,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(columns=['EIN', 'NAME'], axis=1)
application_df.head()


,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [3]:
# Determine the number of unique values in each column.
application_df.nunique()


APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [4]:
# Look at APPLICATION_TYPE value counts for binning
app_counts = application_df['APPLICATION_TYPE'].value_counts()
app_counts


T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [5]:
# Choose a cutoff value and create a list of application types to be replaced
application_types_to_replace = list(app_counts[app_counts < 500].index)

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(
        app, "Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()


T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [6]:
# Look at CLASSIFICATION value counts for binning
class_counts = application_df['CLASSIFICATION'].value_counts()
class_counts


C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [7]:
# You may find it helpful to look at CLASSIFICATION value counts >1
class_counts_gt1 = class_counts.loc[class_counts > 1]
class_counts_gt1.head()


C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [8]:
# Choose a cutoff value and create a list of classifications to be replaced
classifications_to_replace = list(class_counts[class_counts < 1000].index)

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(
        cls, "Other")

# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()


C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [9]:
features = application_df.shape[1]
print("Number of features:", features)

Number of features: 10


In [10]:
# Convert categorical data to numeric with `pd.get_dummies`
application_numeric = pd.get_dummies(application_df)
application_numeric.head()


,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,1,108590,1,0,0,0,1,0,0,0,...,1,0,0,0,0,0,0,0,1,0
2,1,5000,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
3,1,6692,1,0,0,0,1,0,0,0,...,0,1,0,0,0,0,0,0,1,0
4,1,142590,1,0,0,0,1,0,0,0,...,0,0,1,0,0,0,0,0,1,0


In [ ]:
# Plot dummy set
#from sklearn.datasets import make_blobs

#X, y = make_blobs(n_samples=34299, centers=2, n_features=2, random_state=18)

#application_numberic = pd.DataFrame(X, columns=["Feature_1", "Feature_2"])
#application_numeric["Target"] = y

#application_numeric.plot.scatter(x='Feature_1', y='Feature_2', c="Target", colormap='winter')

In [15]:
# Split our preprocessed data into our features and target arrays
X = application_numeric.drop(['IS_SUCCESSFUL'], axis=1)
y = application_numeric['IS_SUCCESSFUL']

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=58)


In [16]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)


## Compile, Train and Evaluate the Model

In [ ]:
# Create a method that creats a new Sequential model with hyperparameter options
def create_model(hp):
    nn_model_2 = tf.keras.models.Sequential()

    # Allow kerastuner to devide which activation function to use in hidden layers
    activation = hp.Choice('activation', ['relu', 'tanh'])

    # Allow kerastuner to decide number of neurons in first layer
    nn_model_2.add(tf.keras.layers.Dense(units=hp.Int('first_units', 
        min_value=1,
        max_value=5,
        step=5), activation=activation, input_shape=(44,)))
    
    # Allow kerastuner to decide number of hidden layers and neurons to add to hidden layers
    for i in range(hp.Int('num_layers', 1, 5)):
        nn_model_2.add(tf.keras.layers.Dense(units=hp.Int('units_' + str(i),
            min_value=1,
            max_value=5,
            step=5),
            activation=activation))
        
    nn_model_2.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

    # Compile the model
    nn_model_2.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

    return nn_model_2

# Create a kerastuner
import keras_tuner as kt

tuner = kt.Hyperband(create_model,
        objective='val_accuracy',
        max_epochs=5,
        hyperband_iterations=3)
                     
# Search for the best hyperparameters
tuner.search(X_train_scaled, y_train, epochs=5, validation_data=(X_test_scaled,y_test))

# Get the best hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

# Build the model with the best hyperparameters
model_2 = create_model(best_hps)

# Train the model
fit_model = model_2.fit(X_train_scaled, y_train, epochs=5)



Trial 28 Complete [00h 00m 18s]
val_accuracy: 0.7210495471954346

Best val_accuracy So Far: 0.7297959327697754
Total elapsed time: 00h 05m 53s
Epoch 1/5
804/804 [==============================] - 4s 3ms/step - loss: 0.6365 - accuracy: 0.6653
Epoch 2/5
804/804 [==============================] - 2s 2ms/step - loss: 0.5861 - accuracy: 0.7230
Epoch 3/5
804/804 [==============================] - 2s 2ms/step - loss: 0.5819 - accuracy: 0.7264
Epoch 4/5
804/804 [==============================] - 2s 3ms/step - loss: 0.5807 - accuracy: 0.7267
Epoch 5/5
804/804 [==============================] - 3s 4ms/step - loss: 0.5796 - accuracy: 0.7271


In [ ]:
# Import the kerastuner library
import keras_tuner as kt

tuner = kt.Hyperband(
    create_model,
    objective='val_accuracy',
    max_epochs=5
    hyperband_iterations=2)

In [ ]:
# Run the kerastuner search for the best hyperparameters
tuner.search(X_train_scaled,y_train,epochs=10,validation_data=(X_test_scaled, y_test))

In [ ]:
# Get best model hyperparameters
best_hyper = tuner.get_best_hyperparameters(1)[0]
for param in best_hyper:
    print(param.values)


In [ ]:
# Build the model with the best hyperparameters
model_2 = create_model(best_hyper)


In [ ]:
# Train the model
fit_model = model_2.fit(X_train_scaled, y_train, epochs=20)

In [ ]:
# Evaluate best model against full test data
model_2 = tuner.get_best_models(1)[0]
model_loss, model_accuracy = model_2.evaluate(X_test_scaled, y_test, verbose=2)
print(f'Loss: {model_loss}, Accuracy: {model_accuracy}')

In [ ]:
# Export our model to HDF5 file
# Define the filename
filename = '/content/drive/MyDrive/UTSA_Homework/Model_H5/AlphabetSoupCharity_Optimization2.h5'

# Save the model to a HDF5 file
model_2.save(filename)
